# Gathering charging station data

### This notebook will document the procedures taken to fetch data about the Norwegian charging stations from OpenChargeMap's API. It also covers the methods used to process the data into a suitable format for further processing and analysis.

In [1]:
import requests
import json
import pandas as pd
import os
import requests
import pyjstat
from pyjstat import pyjstat
import numpy as np

# Display all columns in the df
pd.set_option('display.max_columns', None)
# Set the float format to display numbers in standard decimal notation
pd.options.display.float_format = '{:.5f}'.format

In [4]:
#API user and endpoint information
url="https://api.openchargemap.io/v3"
api_key = "Your API Key Goes HERE!"



In [ ]:
#Fetch
requests.post(url, params={'key': api_key})

In [6]:
#Charging levels: 1 = small charger (13h+ for full charge)
#Charging levels: 2= medimum charger (Can charge 16-120km for an hour), add "3, 2" if you want to test with medium charges as well
#Charging levels: 3= fast charger (full charge in 15min -1.5t)

#Query parameters defined in this step

params = {"countryid": "168",
           "levelid":"3",
           "isoperational":True,
           "output": "json",
           "compact": True,
           "verbose": True,
           "key":api_key,
           "maxresults": 5000,
           "distanceunit": "km"  
           }

#Get data from API here
f = r'https://api.openchargemap.io/v3/poi'
data = requests.get(f, params=params)
a=data.text
a=json.loads(a)

In [ ]:
#How to view the JSON structured data (needed for later function in flattening structure to dataframe)
rep = json.loads(data.text)

# Print the JSON output to the console
print(json.dumps(rep, indent=4))

In [8]:
# Convert the JSON data to a pandas DataFrame
df_a = pd.json_normalize(a, sep="_")

In [9]:
#View data
df_a.head()

,DataProvider,OperatorInfo,UsageType,StatusType,SubmissionStatus,UserComments,PercentageSimilarity,MediaItems,IsRecentlyVerified,DateLastVerified,ID,UUID,ParentChargePointID,DataProviderID,DataProvidersReference,OperatorID,OperatorsReference,UsageTypeID,UsageCost,Connections,NumberOfPoints,GeneralComments,DatePlanned,DateLastConfirmed,StatusTypeID,DateLastStatusUpdate,MetadataValues,DataQualityLevel,DateCreated,SubmissionStatusTypeID,AddressInfo_ID,AddressInfo_Title,AddressInfo_AddressLine1,AddressInfo_AddressLine2,AddressInfo_Town,AddressInfo_StateOrProvince,AddressInfo_Postcode,AddressInfo_CountryID,AddressInfo_Country,AddressInfo_Latitude,AddressInfo_Longitude,AddressInfo_ContactTelephone1,AddressInfo_ContactTelephone2,AddressInfo_ContactEmail,AddressInfo_AccessComments,AddressInfo_RelatedURL,AddressInfo_Distance,AddressInfo_DistanceUnit
0,None,None,None,None,None,None,None,None,True,2023-05-08T04:09:00Z,265371,FE96339B-9482-45FA-9A2D-CEF365EB92C6,None,19,15130,NaN,None,NaN,None,"[{'ID': 450347, 'ConnectionTypeID': 2, 'Connec...",8.00000,None,None,None,0.00000,2023-05-08T04:09:00Z,None,2,2023-05-08T04:09:00Z,100,265755,Recharge Shell Bjerkvik (coming soon),Nordmoveien 17,None,BJERKVIK,Nordland,8530,168,None,68.55121,17.55747,None,None,None,,None,None,0
1,None,None,None,None,None,None,None,None,True,2023-05-08T04:07:00Z,265369,A4E3EC98-998B-43DE-BCFE-F8F7D744A75E,None,19,9533,NaN,None,NaN,None,"[{'ID': 450322, 'ConnectionTypeID': 33, 'Conne...",18.00000,None,None,None,0.00000,2023-05-08T04:07:00Z,None,2,2023-05-08T04:07:00Z,100,265753,"Strandtorget Senter, Lillehammer",Strandpromenaden 85,None,LILLEHAMMER,Innlandet,2609,168,None,61.11446,10.44729,None,None,None,,None,None,0
2,None,None,None,None,None,None,None,None,True,2023-05-08T04:07:00Z,265368,BEA16CB3-E2F5-4052-B483-46D3BDA23B6D,None,19,15128,NaN,None,NaN,None,"[{'ID': 450280, 'ConnectionTypeID': 2, 'Connec...",30.00000,None,None,None,0.00000,2023-05-08T04:07:00Z,None,2,2023-05-08T04:07:00Z,100,265752,Recharge IKEA Åsane (coming soon),Dalavegen 4,None,NYBORG,Vestland,5131,168,None,60.47519,5.33094,None,None,None,,None,None,0
3,None,None,None,None,None,None,None,None,True,2023-05-08T04:06:00Z,265366,3B454C90-911A-4F62-BD5E-650A976A23C5,None,19,15119,NaN,None,NaN,None,"[{'ID': 450267, 'ConnectionTypeID': 33, 'Conne...",9.00000,None,None,None,0.00000,2023-05-08T04:06:00Z,None,2,2023-05-08T04:06:00Z,100,265750,Circle K Nesttun,Nesttunvegen 91,None,NESTTUN,Vestland,5221,168,None,60.32011,5.35146,None,None,None,,None,None,0
4,None,None,None,None,None,None,None,None,True,2023-05-08T04:06:00Z,265365,BFC8E8A7-3F8A-4657-A2BD-91C1B83C8EF3,None,19,4689,NaN,None,NaN,None,"[{'ID': 450237, 'ConnectionTypeID': 25, 'Conne...",20.00000,None,None,None,0.00000,2023-05-08T04:06:00Z,None,2,2023-05-08T04:06:00Z,100,265749,Sven Oftedalsvei 10 Utendørs parkeringsplass,Sven Oftedals vei 10,None,OSLO,Oslo,0950,168,None,59.94345,10.85890,None,None,None,,None,None,0


In [10]:
# Calculate the number of unique IDs in the "ID" column
num_unique_ids = df_a["ID"].nunique()

# Print the result to the console
print("Number of unique IDs:", num_unique_ids)


Number of unique IDs: 1452


## Flattening JSON data-structure

In [11]:
title = []
latitude = []
longitude = []
id= []
num_points = []
status_type = []
power = []
amount = []
comment = []
level = []

for i in range(len(a)):
    temptitle = a[i]['AddressInfo']['Title']
    templat   = a[i]['AddressInfo']['Latitude']
    templon   = a[i]['AddressInfo']['Longitude']
    tempID    = a[i]['AddressInfo']['ID']
    tempNumP  = a[i]['NumberOfPoints']
    
    # iterate over the connections list
    for conn in a[i]['Connections']:
        tempSt_Ty = conn['StatusType']
        temppower = conn['PowerKW']
        tempamount= conn['Quantity']
        tempcoms  = conn['Comments']
        templevel = conn['LevelID']
        
        # append data to the lists
        title.append(temptitle)
        latitude.append(templat)
        longitude.append(templon)
        id.append(tempID)
        num_points.append(tempNumP)
        status_type.append(tempSt_Ty)
        power.append(temppower)
        amount.append(tempamount)
        comment.append(tempcoms)
        level.append(templevel)

d = {'Name':title, 'Latitude':latitude, 'Longitude':longitude, 'ID': id, 'Status_Type': status_type, 'PowerKW': power, 'Quantity': amount, 'Number_of_Points': num_points, 'Mode': comment, 'Level': level}
df = pd.DataFrame(d)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10679 entries, 0 to 10678
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              10679 non-null  object 
 1   Latitude          10679 non-null  float64
 2   Longitude         10679 non-null  float64
 3   ID                10679 non-null  int64  
 4   Status_Type       0 non-null      object 
 5   PowerKW           9747 non-null   float64
 6   Quantity          562 non-null    float64
 7   Number_of_Points  10677 non-null  float64
 8   Mode              10255 non-null  object 
 9   Level             9853 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 834.4+ KB


In [13]:
df.head()

,Name,Latitude,Longitude,ID,Status_Type,PowerKW,Quantity,Number_of_Points,Mode,Level
0,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8.00000,Mode 4,3.00000
1,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,150.00000,NaN,8.00000,Mode 4,3.00000
2,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8.00000,Mode 4,3.00000
3,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,150.00000,NaN,8.00000,Mode 4,3.00000
4,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,NaN,NaN,8.00000,Mode 4,NaN


In [14]:
#Convert floats to integers where applicable

df['Number_of_Points'] = df['Number_of_Points'].replace([np.inf, -np.inf], np.nan).astype('Int64')
df['Level'] = df['Level'].replace([np.inf, -np.inf], np.nan).astype('Int64')


In [15]:
df.head()


,Name,Latitude,Longitude,ID,Status_Type,PowerKW,Quantity,Number_of_Points,Mode,Level
0,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8,Mode 4,3
1,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,150.00000,NaN,8,Mode 4,3
2,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8,Mode 4,3
3,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,150.00000,NaN,8,Mode 4,3
4,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,NaN,NaN,8,Mode 4,<NA>


In [16]:
#What type of power do these chargers output?

df_counts = df['PowerKW'].value_counts().sort_index(ascending=False)
print(df_counts)


350.00000     106
300.00000       1
250.00000       1
200.00000       1
180.00000       3
175.00000       2
150.00000    1161
135.00000      50
129.00000      11
125.00000       1
120.00000      54
100.00000     151
75.00000        3
62.50000        5
62.00000        4
60.00000        3
55.00000        2
50.00000     2744
48.00000        4
44.00000       32
43.00000      305
25.00000       11
22.00000     4365
20.00000       82
13.00000        2
12.00000      113
11.10000        3
11.00000      198
7.40000       145
7.00000         7
3.70000       174
3.68000         2
3.60000         1
Name: PowerKW, dtype: int64


In [17]:
# Only keep chargers with efficacy of above 25 kW (meaning the lowest performance charger will be 43 kW)
df_filtered = df[df['PowerKW'] > 25]

# Calculate the value counts of the filtered DataFrame and sort them in descending order
df_counts_filtered = df_filtered['PowerKW'].value_counts().sort_index(ascending=False)

# Print the filtered value counts
print(df_counts_filtered)

350.00000     106
300.00000       1
250.00000       1
200.00000       1
180.00000       3
175.00000       2
150.00000    1161
135.00000      50
129.00000      11
125.00000       1
120.00000      54
100.00000     151
75.00000        3
62.50000        5
62.00000        4
60.00000        3
55.00000        2
50.00000     2744
48.00000        4
44.00000       32
43.00000      305
Name: PowerKW, dtype: int64


In [18]:
#Remove these low power stations

# Drop rows where PowerKW is less than 22
df.drop(df[df['PowerKW'] < 43].index, inplace=True)

In [19]:
df_counts = df['Level'].value_counts()
print(df_counts)

3    4712
2      38
Name: Level, dtype: Int64


### Create a QGIS geospatial compatible data column called "geometry"

In [20]:
#Create geometry column in dataframe

from shapely.geometry import Point

# create a list of Point objects
points = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]

# create a new column 'geometry' in the dataframe with the Point objects
df['geometry'] = points


In [21]:
df.head()

,Name,Latitude,Longitude,ID,Status_Type,PowerKW,Quantity,Number_of_Points,Mode,Level,geometry
0,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8,Mode 4,3,POINT (17.557469 68.551208)
1,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,150.00000,NaN,8,Mode 4,3,POINT (17.557469 68.551208)
2,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8,Mode 4,3,POINT (17.557469 68.551208)
3,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,150.00000,NaN,8,Mode 4,3,POINT (17.557469 68.551208)
4,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,NaN,NaN,8,Mode 4,<NA>,POINT (17.557469 68.551208)


In [22]:
#Removing observations where Level or Power is equal to zero or blanks

df = df.dropna(subset=['Level', 'PowerKW'])


In [23]:
df_counts = df['Level'].value_counts()
print(df_counts)

3    4607
2      37
Name: Level, dtype: Int64


In [24]:
df_grouped = df.groupby('ID')['PowerKW'].agg(list).reset_index()            # Group the dataframe by ID and aggregate the values of PowerKW as a list
df_grouped = df_grouped.rename(columns={'PowerKW': 'PowerKW_list'})         # Rename the column to PowerKW_list
df = df.merge(df_grouped, on='ID')                                          # Merge the original dataframe with the grouped dataframe to add the new column
df = df.drop_duplicates(subset='ID')                                        # Drop duplicates based on the ID column
df                                                                          # Display the final dataframe

,Name,Latitude,Longitude,ID,Status_Type,PowerKW,Quantity,Number_of_Points,Mode,Level,geometry,PowerKW_list
0,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8,Mode 4,3,POINT (17.557469 68.551208),"[50.0, 150.0, 50.0, 150.0]"
4,"Strandtorget Senter, Lillehammer",61.11446,10.44729,265753,None,150.00000,NaN,18,Mode 4,3,POINT (10.447287368802282 61.114455202428466),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50...."
14,Recharge IKEA Åsane (coming soon),60.47519,5.33094,265752,None,50.00000,NaN,30,Mode 4,3,POINT (5.330944 60.475191),"[50.0, 150.0, 50.0, 150.0, 150.0, 150.0, 50.0,..."
24,Circle K Nesttun,60.32011,5.35146,265750,None,150.00000,NaN,9,Mode 4,3,POINT (5.35146 60.32011),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50...."
33,Strandgata 25,70.37330,31.11009,262254,None,50.00000,NaN,4,Mode 4,3,POINT (31.110092 70.373297),"[50.0, 50.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4637,Dombås Supercharger,62.07493,9.12828,21063,None,120.00000,16.00000,16,None,3,POINT (9.128281 62.074928),[120.0]
4638,Lyngdal Supercharger,58.15652,7.10296,19381,None,135.00000,8.00000,20,None,3,POINT (7.10296 58.15652),[135.0]
4639,Lillehammer Supercharger,61.13046,10.43492,19331,None,135.00000,8.00000,14,None,3,POINT (10.434918 61.130464),[135.0]
4640,Fortum Hurtigladestasjon Værnes Lufthavn,63.45122,10.91955,10577,None,50.00000,3.00000,9,Mode 4,3,POINT (10.919549 63.451216),"[50.0, 50.0]"


In [25]:
# Reset the index
df = df.reset_index(drop=True)
# # Drop the column named "index"
# df = df.drop('index', axis=1)
df.head()

,Name,Latitude,Longitude,ID,Status_Type,PowerKW,Quantity,Number_of_Points,Mode,Level,geometry,PowerKW_list
0,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,None,50.00000,NaN,8,Mode 4,3,POINT (17.557469 68.551208),"[50.0, 150.0, 50.0, 150.0]"
1,"Strandtorget Senter, Lillehammer",61.11446,10.44729,265753,None,150.00000,NaN,18,Mode 4,3,POINT (10.447287368802282 61.114455202428466),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50...."
2,Recharge IKEA Åsane (coming soon),60.47519,5.33094,265752,None,50.00000,NaN,30,Mode 4,3,POINT (5.330944 60.475191),"[50.0, 150.0, 50.0, 150.0, 150.0, 150.0, 50.0,..."
3,Circle K Nesttun,60.32011,5.35146,265750,None,150.00000,NaN,9,Mode 4,3,POINT (5.35146 60.32011),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50...."
4,Strandgata 25,70.37330,31.11009,262254,None,50.00000,NaN,4,Mode 4,3,POINT (31.110092 70.373297),"[50.0, 50.0]"


In [26]:
#Dropping redundant variables

df = df.drop('PowerKW', axis=1)
df = df.drop('Quantity', axis=1)
df = df.drop('Status_Type', axis=1)

df.head()

,Name,Latitude,Longitude,ID,Number_of_Points,Mode,Level,geometry,PowerKW_list
0,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,8,Mode 4,3,POINT (17.557469 68.551208),"[50.0, 150.0, 50.0, 150.0]"
1,"Strandtorget Senter, Lillehammer",61.11446,10.44729,265753,18,Mode 4,3,POINT (10.447287368802282 61.114455202428466),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50...."
2,Recharge IKEA Åsane (coming soon),60.47519,5.33094,265752,30,Mode 4,3,POINT (5.330944 60.475191),"[50.0, 150.0, 50.0, 150.0, 150.0, 150.0, 50.0,..."
3,Circle K Nesttun,60.32011,5.35146,265750,9,Mode 4,3,POINT (5.35146 60.32011),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50...."
4,Strandgata 25,70.37330,31.11009,262254,4,Mode 4,3,POINT (31.110092 70.373297),"[50.0, 50.0]"


In [27]:
#Update the Number_Of_Points after removing the low power charging stations
df = df.drop('Number_of_Points', axis=1)

# Define a lambda function to calculate the length of the list
count_chargers = lambda x: len(x)

# Apply the lambda function to the PowerKW_list column to create the Number_Of_Chargers column
df['Number_Of_Chargers'] = df['PowerKW_list'].apply(count_chargers)


In [28]:
df.head()

,Name,Latitude,Longitude,ID,Mode,Level,geometry,PowerKW_list,Number_Of_Chargers
0,Recharge Shell Bjerkvik (coming soon),68.55121,17.55747,265755,Mode 4,3,POINT (17.557469 68.551208),"[50.0, 150.0, 50.0, 150.0]",4
1,"Strandtorget Senter, Lillehammer",61.11446,10.44729,265753,Mode 4,3,POINT (10.447287368802282 61.114455202428466),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50....",10
2,Recharge IKEA Åsane (coming soon),60.47519,5.33094,265752,Mode 4,3,POINT (5.330944 60.475191),"[50.0, 150.0, 50.0, 150.0, 150.0, 150.0, 50.0,...",10
3,Circle K Nesttun,60.32011,5.35146,265750,Mode 4,3,POINT (5.35146 60.32011),"[150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 50....",9
4,Strandgata 25,70.37330,31.11009,262254,Mode 4,3,POINT (31.110092 70.373297),"[50.0, 50.0]",2


In [ ]:
#Converting dataframe into QGIS compatible dataframe : different methods

#Shapefile method
import geopandas as gpd

# # convert the DataFrame to a GeoDataFrame
# gdf = gpd.GeoDataFrame(df, crs='EPSG:5939', geometry='geometry')

# # save the GeoDataFrame as a shapefile
# gdf.to_file('updated_cs_stations.shp', driver='ESRI Shapefile')



# # create a GeoDataFrame from the DataFrame and geometry
# gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:5939')

# # save the GeoDataFrame as a GeoJSON file
# gdf.to_file('cs.geojson', driver='GeoJSON')

#Excel method
df.to_csv("updated_charging_stations_2.csv", header=True)

# After work in QGIS on updated_charging_stations


### After the charging stations have been merged with the roads from the network and the data exported back

In [30]:
import geopandas as gpd

# read the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(r'C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\From_Mac\for_qgis\merged_Road_CS_43up.geojson')



In [31]:
gdf.head()

,source,target,length,oneway,grade_abs,roadclass,name,ref,Name_2,ID_2,Number_Of_Chargers,n,distance,feature_x,feature_y,nearest_x,nearest_y,geometry
0,794436,735703,11865.76308,"[False, False, False, False, False, False, Fal...","[ 0.999, 1.4370000000000001, 0.258000000000000...","[0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, ...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...",Alna senter,243288.00000,6.00000,1.00000,0.00000,10.84593,59.92731,10.84593,59.92730,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
1,794436,735703,11865.76308,"[False, False, False, False, False, False, Fal...","[ 0.999, 1.4370000000000001, 0.258000000000000...","[0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, ...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...","Circle K Økern Hurtigladestasjon, Oslo",46650.00000,2.00000,2.00000,0.00001,10.81487,59.92251,10.81487,59.92250,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
2,794436,735703,11865.76308,"[False, False, False, False, False, False, Fal...","[ 0.999, 1.4370000000000001, 0.258000000000000...","[0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, ...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...","Fortum hurtigladestasjon Kiwi Bjerke, Oslo",46721.00000,2.00000,3.00000,0.00003,10.81317,59.92225,10.81318,59.92222,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
3,794436,735703,11865.76308,"[False, False, False, False, False, False, Fal...","[ 0.999, 1.4370000000000001, 0.258000000000000...","[0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, ...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...",Recharge McDonald´s Alnabru,243279.00000,6.00000,4.00000,0.00004,10.84123,59.92658,10.84123,59.92654,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
4,794436,735703,11865.76308,"[False, False, False, False, False, False, Fal...","[ 0.999, 1.4370000000000001, 0.258000000000000...","[0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, ...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...","[E6, E18, E6, E18, E6, Jerikoveien, E6, Rv4, R...",Obs Haugenstua,157384.00000,10.00000,5.00000,0.00005,10.90227,59.93600,10.90226,59.93605,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"


In [ ]:
#Cleaning the dataset

# drop multiple columns from the DataFrame
gdf = gdf.drop(['n', 'distance', 'feature_x', 'feature_y', 'nearest_x', 'nearest_y', 'Number_Of_Chargers'], axis=1)
#Dropping more unnecessary columns
gdf = gdf.drop(['oneway', 'grade_abs', 'roadclass', 'name', 'ref'], axis=1)

# print the updated DataFrame
print(gdf.columns)



In [39]:
gdf.head()

,source,target,length,Name_2,ID_2,geometry
0,794436,735703,11865.76308,Alna senter,243288.00000,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
1,794436,735703,11865.76308,"Circle K Økern Hurtigladestasjon, Oslo",46650.00000,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
2,794436,735703,11865.76308,"Fortum hurtigladestasjon Kiwi Bjerke, Oslo",46721.00000,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
3,794436,735703,11865.76308,Recharge McDonald´s Alnabru,243279.00000,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
4,794436,735703,11865.76308,Obs Haugenstua,157384.00000,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"


In [40]:
# rename the columns
gdf = gdf.rename(columns={'ID_2': 'CS_id'})

gdf['CS_id'] = gdf['CS_id'].fillna(0)

# convert the 'age' column from float to int
gdf['CS_id'] = gdf['CS_id'].astype(int)

In [41]:
gdf.head()

,source,target,length,Name_2,CS_id,geometry
0,794436,735703,11865.76308,Alna senter,243288,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
1,794436,735703,11865.76308,"Circle K Økern Hurtigladestasjon, Oslo",46650,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
2,794436,735703,11865.76308,"Fortum hurtigladestasjon Kiwi Bjerke, Oslo",46721,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
3,794436,735703,11865.76308,Recharge McDonald´s Alnabru,243279,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"
4,794436,735703,11865.76308,Obs Haugenstua,157384,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)"


## Merge datasets


In [42]:
sub_df = df.loc[:, ['ID', 'Level', 'PowerKW_list', 'Number_Of_Chargers']]
sub_df = sub_df.rename(columns={'ID': 'CS_id'})


In [43]:
# join the two DataFrames by 'CS_id'
merged_df = pd.merge(gdf, sub_df, on='CS_id', how='left')

In [44]:
merged_df

,source,target,length,Name_2,CS_id,geometry,Level,PowerKW_list,Number_Of_Chargers
0,794436,735703,11865.76308,Alna senter,243288,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0]",6.00000
1,794436,735703,11865.76308,"Circle K Økern Hurtigladestasjon, Oslo",46650,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0]",2.00000
2,794436,735703,11865.76308,"Fortum hurtigladestasjon Kiwi Bjerke, Oslo",46721,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[44.0, 44.0]",2.00000
3,794436,735703,11865.76308,Recharge McDonald´s Alnabru,243279,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0, 50.0, 50.0, 150.0, 150.0]",6.00000
4,794436,735703,11865.76308,Obs Haugenstua,157384,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50....",10.00000
...,...,...,...,...,...,...,...,...,...
1751,973930,1818925,15500.51499,NaN,0,"LINESTRING (26.56513 70.34171, 26.85529 70.44074)",<NA>,NaN,NaN
1752,1818925,832749,17778.22677,Fjellvåkveien 15,217034,"LINESTRING (26.85529 70.44074, 27.33086 70.44362)",3,"[100.0, 150.0, 50.0, 50.0]",4.00000
1753,832749,1101914,17561.22246,NaN,0,"LINESTRING (27.33086 70.44362, 27.80045 70.43863)",<NA>,NaN,NaN
1754,1101914,1007500,15461.23499,NaN,0,"LINESTRING (27.80045 70.43863, 28.19107 70.39350)",<NA>,NaN,NaN


In [ ]:
na_counts = merged_df.isna().sum()
print(na_counts)

merged_df['Number_Of_Chargers'] = merged_df['Number_Of_Chargers'].astype(int)

In [47]:
merged_df['Name'] = merged_df['Name_2'].fillna("No CS")
merged_df['Level'] = merged_df['Level'].fillna(0)
merged_df['PowerKW_list'] = merged_df['PowerKW_list'].fillna("No CS")
merged_df['Number_Of_Chargers'] = merged_df['Number_Of_Chargers'].fillna(0)

In [51]:
#merged dataframe
merged_df

merged_df = merged_df.drop("Name_2", axis=1)

In [52]:
merged_df

,source,target,length,CS_id,geometry,Level,PowerKW_list,Number_Of_Chargers,Name
0,794436,735703,11865.76308,243288,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0]",6.00000,Alna senter
1,794436,735703,11865.76308,46650,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0]",2.00000,"Circle K Økern Hurtigladestasjon, Oslo"
2,794436,735703,11865.76308,46721,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[44.0, 44.0]",2.00000,"Fortum hurtigladestasjon Kiwi Bjerke, Oslo"
3,794436,735703,11865.76308,243279,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0, 50.0, 50.0, 150.0, 150.0]",6.00000,Recharge McDonald´s Alnabru
4,794436,735703,11865.76308,157384,"LINESTRING (10.73379 59.90999, 10.93659 59.94130)",3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50....",10.00000,Obs Haugenstua
...,...,...,...,...,...,...,...,...,...
1751,973930,1818925,15500.51499,0,"LINESTRING (26.56513 70.34171, 26.85529 70.44074)",0,No CS,0.00000,No CS
1752,1818925,832749,17778.22677,217034,"LINESTRING (26.85529 70.44074, 27.33086 70.44362)",3,"[100.0, 150.0, 50.0, 50.0]",4.00000,Fjellvåkveien 15
1753,832749,1101914,17561.22246,0,"LINESTRING (27.33086 70.44362, 27.80045 70.43863)",0,No CS,0.00000,No CS
1754,1101914,1007500,15461.23499,0,"LINESTRING (27.80045 70.43863, 28.19107 70.39350)",0,No CS,0.00000,No CS


In [ ]:
merged_traffic=pd.read_csv(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\From_Mac\Data\problem.csv")
merged_traffic

In [63]:
merged_traffic = merged_traffic.drop(["Unnamed: 0.1", "Unnamed: 0","u","v"],axis=1)

In [64]:
merged_traffic

,source,target,length,CS_id,geometry,Level,PowerKW_list,Number_Of_Chargers,Name,traffic_id
0,794436,735703,11865.76308,243288,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0]",6.00000,Alna senter,59192V1878201
1,794436,735703,11865.76308,46650,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0]",2.00000,"Circle K Økern Hurtigladestasjon, Oslo",59192V1878201
2,794436,735703,11865.76308,46721,LINESTRING (10.733792828475945 59.909993840978...,3,"[44.0, 44.0]",2.00000,"Fortum hurtigladestasjon Kiwi Bjerke, Oslo",59192V1878201
3,794436,735703,11865.76308,243279,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0, 50.0, 50.0, 150.0, 150.0]",6.00000,Recharge McDonald´s Alnabru,59192V1878201
4,794436,735703,11865.76308,157384,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50....",10.00000,Obs Haugenstua,59192V1878201
...,...,...,...,...,...,...,...,...,...,...
1751,973930,1818925,15500.51499,0,LINESTRING (26.565130981516866 70.341708562793...,0,No CS,0.00000,No CS,14490V930350
1752,1818925,832749,17778.22677,217034,LINESTRING (26.85528555460236 70.4407373056435...,3,"[100.0, 150.0, 50.0, 50.0]",4.00000,Fjellvåkveien 15,14490V930350
1753,832749,1101914,17561.22246,0,LINESTRING (27.33086343693628 70.4436216410368...,0,No CS,0.00000,No CS,14490V930350
1754,1101914,1007500,15461.23499,0,LINESTRING (27.80045221095047 70.4386256661820...,0,No CS,0.00000,No CS,14490V930350


In [61]:
# Find rows where "traffic_id" is NaN
nan_rows = merged_traffic[merged_traffic["traffic_id"].isnull()]

# Print the rows where "traffic_id" is NaN
nan_rows

# # Count the occurrences of NaN in the "traffic_id" column
# nan_count = merged_traffic["traffic_id"].isnull().value_counts()

# nan_count

,source,target,length,CS_id,geometry,Level,PowerKW_list,Number_Of_Chargers,Name,traffic_id,volume,1,2,3,4,5,6,7,8,9,10,11,12
996,298536,943350,10011.83935,0,LINESTRING (6.084103686808975 62.3010377867154...,0,No CS,0.00000,No CS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,943350,1220613,11222.53052,157527,LINESTRING (6.235154726998502 62.3570273887099...,3,"[50.0, 50.0]",2.00000,Ålesund Moa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,943350,1220613,11222.53052,127389,LINESTRING (6.235154726998502 62.3570273887099...,3,[50.0],1.00000,Borgundvegen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999,943350,1220613,11222.53052,216513,LINESTRING (6.235154726998502 62.3570273887099...,3,"[50.0, 150.0, 50.0, 150.0, 150.0, 150.0, 150.0...",8.00000,Recharge Esso Spjelkavik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,943350,1220613,11222.53052,216511,LINESTRING (6.235154726998502 62.3570273887099...,3,"[50.0, 50.0]",2.00000,Recharge Shell Vegsund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,943350,1220613,11222.53052,173582,LINESTRING (6.235154726998502 62.3570273887099...,3,"[150.0, 150.0]",2.00000,Brusdalsvegen 201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1392,1095954,1594521,15613.69358,173681,LINESTRING (28.261649002416984 70.168640371306...,3,"[50.0, 50.0, 100.0, 150.0]",4.00000,Deatnomielli -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1755,1007500,1594521,10025.49048,0,LINESTRING (28.19107356742026 70.3935023188165...,0,No CS,0.00000,No CS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
#Add all months volume data

volume_df = pd.read_csv(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\Statistics\ultra_simplified\all_road_volume_org.csv")
volume_df = volume_df[['id', 'name', 'month', 'volume']] # select columns of interest
volume_df.drop_duplicates(subset=['id', 'month'], inplace=True) # remove duplicates
volume_df_pivoted = volume_df.pivot(index='id', columns='month', values='volume') # pivot the data
volume_df_pivoted


month,1,2,3,4,5,6,7,8,9,10,11,12
id,,,,,,,,,,,,
00000V1702725,10002.00000,9596.00000,9765.00000,10333.00000,10260.00000,11587.00000,10288.00000,11123.00000,10266.00000,10855.00000,11788.00000,12356.00000
00000V1702751,35838.00000,34800.00000,33886.00000,39673.00000,40976.00000,44202.00000,42728.00000,42922.00000,39273.00000,40556.00000,40479.00000,37035.00000
00000V1993681,23810.00000,23809.00000,21952.00000,23258.00000,22205.00000,24293.00000,17966.00000,12962.00000,13526.00000,13945.00000,14080.00000,11689.00000
00000V443295,45704.00000,48198.00000,48216.00000,52830.00000,54808.00000,61031.00000,55277.00000,59859.00000,56073.00000,54632.00000,52830.00000,47071.00000
00000V443437,3412.00000,3733.00000,3925.00000,4093.00000,4210.00000,4689.00000,4612.00000,4784.00000,4658.00000,4504.00000,4160.00000,3714.00000
...,...,...,...,...,...,...,...,...,...,...,...,...
99915V578630,2985.00000,3106.00000,3116.00000,3426.00000,3513.00000,3749.00000,3282.00000,3675.00000,3494.00000,3529.00000,3430.00000,3098.00000
99923V578123,176.00000,230.00000,337.00000,204.00000,232.00000,287.00000,384.00000,365.00000,255.00000,219.00000,203.00000,193.00000
99950V704686,1601.00000,1670.00000,1668.00000,1865.00000,2076.00000,2149.00000,1811.00000,2037.00000,2051.00000,1962.00000,1875.00000,1642.00000


In [66]:
new_gdf = pd.merge(merged_traffic, volume_df_pivoted, how = 'left', left_on='traffic_id', right_on='id')
new_gdf

,source,target,length,CS_id,geometry,Level,PowerKW_list,Number_Of_Chargers,Name,traffic_id,1,2,3,4,5,6,7,8,9,10,11,12
0,794436,735703,11865.76308,243288,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0]",6.00000,Alna senter,59192V1878201,34276.00000,35539.00000,35016.00000,38122.00000,37843.00000,41610.00000,30198.00000,38625.00000,38102.00000,39098.00000,38699.00000,33801.00000
1,794436,735703,11865.76308,46650,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0]",2.00000,"Circle K Økern Hurtigladestasjon, Oslo",59192V1878201,34276.00000,35539.00000,35016.00000,38122.00000,37843.00000,41610.00000,30198.00000,38625.00000,38102.00000,39098.00000,38699.00000,33801.00000
2,794436,735703,11865.76308,46721,LINESTRING (10.733792828475945 59.909993840978...,3,"[44.0, 44.0]",2.00000,"Fortum hurtigladestasjon Kiwi Bjerke, Oslo",59192V1878201,34276.00000,35539.00000,35016.00000,38122.00000,37843.00000,41610.00000,30198.00000,38625.00000,38102.00000,39098.00000,38699.00000,33801.00000
3,794436,735703,11865.76308,243279,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0, 50.0, 50.0, 150.0, 150.0]",6.00000,Recharge McDonald´s Alnabru,59192V1878201,34276.00000,35539.00000,35016.00000,38122.00000,37843.00000,41610.00000,30198.00000,38625.00000,38102.00000,39098.00000,38699.00000,33801.00000
4,794436,735703,11865.76308,157384,LINESTRING (10.733792828475945 59.909993840978...,3,"[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50....",10.00000,Obs Haugenstua,59192V1878201,34276.00000,35539.00000,35016.00000,38122.00000,37843.00000,41610.00000,30198.00000,38625.00000,38102.00000,39098.00000,38699.00000,33801.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,973930,1818925,15500.51499,0,LINESTRING (26.565130981516866 70.341708562793...,0,No CS,0.00000,No CS,14490V930350,251.00000,305.00000,342.00000,367.00000,438.00000,554.00000,858.00000,683.00000,488.00000,361.00000,323.00000,260.00000
1752,1818925,832749,17778.22677,217034,LINESTRING (26.85528555460236 70.4407373056435...,3,"[100.0, 150.0, 50.0, 50.0]",4.00000,Fjellvåkveien 15,14490V930350,251.00000,305.00000,342.00000,367.00000,438.00000,554.00000,858.00000,683.00000,488.00000,361.00000,323.00000,260.00000
1753,832749,1101914,17561.22246,0,LINESTRING (27.33086343693628 70.4436216410368...,0,No CS,0.00000,No CS,14490V930350,251.00000,305.00000,342.00000,367.00000,438.00000,554.00000,858.00000,683.00000,488.00000,361.00000,323.00000,260.00000
1754,1101914,1007500,15461.23499,0,LINESTRING (27.80045221095047 70.4386256661820...,0,No CS,0.00000,No CS,14490V930350,251.00000,305.00000,342.00000,367.00000,438.00000,554.00000,858.00000,683.00000,488.00000,361.00000,323.00000,260.00000


In [67]:
# Find rows where "traffic_id" is NaN
nan_rows = new_gdf[new_gdf["traffic_id"].isnull()]

# Print the rows where "traffic_id" is NaN
nan_rows

,source,target,length,CS_id,geometry,Level,PowerKW_list,Number_Of_Chargers,Name,traffic_id,1,2,3,4,5,6,7,8,9,10,11,12


# End resulting dataframe to be used in model

In [69]:

new_gdf.to_excel(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\From_Mac\Data\CS_Volume_Roads.xlsx", header=True)